# Solving the Poisson problem on CPU and GPU - The 5-point stencil

In [ ]:
import numpy as np
import numba
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
import numba
from numba import cuda
import math
np.set_printoptions(threshold=np.inf)
%matplotlib inline

### Evaluating the discrete Laplacian on CPUs

We consider the following Poisson problem on the unit square $\Omega = [0, 1]^2$. Solve

$$
-\Delta u = 1
$$

in $\Omega$ with $x = 0$ on $\partial\Omega$. To solve this problem we consider a discretisation based on the finite difference method. Let $x_i = ih$ with $h = \frac{1}{N - 1}$ and $i=0, \dots, N - 1$. Let $y_j$ be defined similarly. We want to compute approximate solutions $u$ at the points $(x_i, y_j)$. Hence, define

$$
u_{i,j} := u(x_i, y_j).
$$

We now approximate the value $-\Delta u(x_i, y_j)$ by the following approximation.

$$
-\Delta u_{i, j}\approx \frac{4u_{i, j} - u_{i - 1, j} - u_{i + 1, j} - u_{i, j - 1} - u_{i, j+ 1}}{h^2}
$$

for interior points $0 < i, j < N - 1$. On boundary points we impose the condition $u_{i, j} = 0$. With this discretisation the PDE problem becomes a linear matrix problem

$$
A x = f
$$

with $f$ being a vector of all ones for interior nodes and zero at the boundary and the vector $x\in\mathbb{R}^{N^2}$ collects all the values $u_{i, j}$ such that $u_{i, j} = x_{jN + i}$.

We now want to implement the generation of the associated matrix. Let us calculate how many nonzero matrix elements we have. We have $(N - 2)^2$ interior points. Each row of $A$ associated with an interior point has $5$ entries. Furthermore, we have $4N - 4$ boundary points. Each boundary point requires a single diagonal entry in the matrix $A$. In total we therefore have

$$
5 (N^2 - 4N + 4) + 4N - 4 = 5N^2 - 16N + 16
$$

nonzero entries in the matrix $A$. The matrix $A$ has dimension $N^2$. Hence, there are in total $N^4$ elements. This means that the fraction of nonzero elements to overall elements is of order $O(N^{-2})$, or in other words. If we have $N = 100$ then the overall matrix dimension is $10,000$ and the number of nonzero elements is of the order of one tenthousands of the total number of matrix elements. The following is Python code that generates the matrix $A$ and the right-hand side $f$, and then solves the linear system and visualizes the solution.


In [ ]:
def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""
    
    nelements = 5 * N**2 - 16 * N + 16
    
    row_ind = np.empty(nelements, dtype=np.float32)
    col_ind = np.empty(nelements, dtype=np.float32)
    data = np.empty(nelements, dtype=np.float32)
    
    f = np.empty(N * N, dtype=np.float32)
    
    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  1
                f[j * N + i] = 0
                count += 1
                
            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i
                                
                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1
                
                count += 5
                                                
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

### Evaluating the discrete Laplacian on GPUs

Here we create a GPU kernel, which given a one-dimensional array of values $u_{i,j}$ in the unit square grid, evaluates this discrete Laplace operator without explicity creating a matrix. Instead $N^2$ threads check if its associated with a boundary value or an interior value. If it is associated with a boundary value, the kernal writes the corresponding $u_{i,j}$ value in the result array (because our sparse matrix was just the identity for boundary values). If it is associated with an interior point, the kernal writes the value of evaluating the 5 point stencil for the corresponding interior point into the result array.

We know that the first and last row in the 2D representation of $u_{i,j}$ correspond to boundry values. For (1D) thread position $x$, this condition can is satisfied if:
- $x < (N - 1)$,  or 
- $N^2 - N < x \leq N^2 - 1)$

We also know that the first and last column in the 2D representation of $u_{i,j}$ correspond to boundry values. For (1D) thread position $x$, this condition can is satisfied if:
- $x$ is divisible by $N$ (leftmost column), or 
- $x + 1$ is divisibile by $N$ (rightmost column)

We know that the each point in the 2D representation of $u_{i,j}$ will have a corresponding a five point stencil in 1D given by: 
- $u(x)$ for the point itself
- $u(x-1)$ for the point on its left
- $u(x+1)$ for the point on its right
- $u(x-N)$ for the point above it
- $u(x+N)$ for the point below it

The equation for $-\Delta u_{i, j}$ can then be used to calculate non-boundry values.

In [ ]:
# CUDA implementation of discrete laplacian operator without shared memory (see next section for shared memory)

@cuda.jit
def discretise_poisson_gpu(vec_in, vec_out):
    '''
    Evaluate the linear matrix problem: Ax = f, 
    without the need to define the matrix A.
    
    Keyword arguments:
    vec_in  -- ndarray of shape [N^2,], corresponding to the solution at each point
    vec_out  -- ndarray of shape [N^2,], corresponding to the interior nodes
    
    '''
    
    i, j = cuda.grid(2)

    if i >= N:
        return
    if j >= N:
        return
    
    # Compute the vector index
    k = j * N + i

    if i == 0 or i == N - 1 or j == 0 or j == N - 1:
        # We are at the boundary
        # Here, the matrix just acts like the identity
        vec_out[k] = vec_in[k]
        return

    # Now deal with the interior element
    up = vec_in[(j + 1) * N + i]
    down = vec_in[(j - 1) * N + i]
    left = vec_in[j * N + i - 1]
    right = vec_in[j * N + i + 1]
    center = vec_in[k]

    vec_out[k] = (N - 1)**2 * (numba.float32(4) * center - up - down - left - right)

    
def eval_gpu(x):
    "Evaluate the discrete Laplacian on the GPU."

    res = np.empty(N * N, dtype=np.float32)

    nblocks = (N + 31) // 32
    discretise_poisson_gpu[(nblocks, nblocks), (32, 32)](x.astype('float32'), res)
    
    return res.astype('float64')

Here we validate the above kernal against a pre-written CPU kernal. Given the problem attempts to solve:
$$
-\Delta u = 1
$$
we expect the above kernal to return the array $f_{ij} = 1$ for all interior nodes and $f_{ij} = 0$ for boundry values. I.e. we expect $f$ (when reshaped to $N$ by $N$) to be the matrix:

$$
\begin{matrix}
0  &  0  &  0  &\ldots & 0\\
0  &  1  &  1  &\ldots & 0\\
0  &  1  &  1  &\ldots & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots\\
0  &   0  & 0     &\ldots & 0
\end{matrix}
$$

In [ ]:
# Validation

# Define padder (pad boundries with zeros)
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 0)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value

rand = np.random.RandomState(0)

# Parameters of linear equation
N = 1000
omega = 0

# Function to discretize
u =  np.pad(rand.randn(N-2, N-2), 1, pad_with)
    
# Array of interior points
u_tilda = u[1:N-1, 1:N-1].flatten()

# CPU calculation
A, _ = discretise_poisson(N)
f_cpu_flat = A @ u.flatten()
f_cpu_square = f_cpu_flat.reshape(N, N)

# GPU calculation
f_gpu_interior_flat = eval_gpu(u_tilda)
f_gpu_interior_square = f_gpu_interior_flat.reshape(N-2, N-2)
f_gpu_square = np.pad(f_gpu_interior_square, 1, pad_with)

rel_error = np.linalg.norm(f_cpu_square.flatten() - f_gpu_square.flatten(), np.inf) / np.linalg.norm(f_cpu_square.flatten(), np.inf)
print(f"Relative error: {rel_error}.")

We can see that the error is in the order of machine precision in 32 bit floating point accuracy. This is because on the CPU we are using double precision numbers and on the GPU we use single precision numbers. 

### Improving the 5-point stencil in CUDA 

The large $N$ case is narrowly faster on the GPU than on the CPU. The problem is memory transfer. For interior points we have 6 accesses to global memory (5 reads, one write) and 6 floating point operations. Hence, the kernels run mostly memory bound. 

To not underutilize the device, we need to load more data than threads. We read blocks of data into shared memory that are spatially close by. The kernels within a workgroup can then locally work on these with much reduced memory accesses.

We need to be careful of boundry conditions since the 5 point stencil structure reaches above, below, left and right of each point. Hence, we must first load data into the _interior_ threads of each block, i.e. we load data into the internal thread points: $[i+1][j+1]$. This will leave a *'halo'* surrounding each block, for which the rows and columns of adjescent blocks can be loaded in, ensuring the stencil calculation is always well defined for data centered on $[i+1][j+1]$.

We also need to be careful setting the global boundry values explicitly to zero. That is, when the global thread position is on the mesh boundry, we load in zero. For each boundary point $x_k$ we always have $x_k = 0$. By doing this, we can therefore reduce the system to something that only acts on the interior grid points, giving us an operator $\tilde{A}:\rightarrow \mathbb{R}^{(N-2)^2}\rightarrow\mathbb{R}^{(N-2)^2}$ where the vector $\tilde{x}$ when we apply $\tilde{A}\tilde{x}$ only contains the interior points $u_{i, j}$ for $0 < i, j < N$.

We also allow a larger class of possible PDE problems, namely so called modified Helmholtz problems of the form

$$
-\Delta u + \omega^2 u = f
$$

by introducing an aditional input parameter $\omega$ (see Helmholtz notebook).

In [4]:
# CUDA implementation of discrete laplacian operator with shared memory

# Thread block size of interior points
SX = 32 
SY = 32

# Total thread block size including 'halo' boundries
haloSX = SX + 2
haloSY = SY + 2

@cuda.jit
def discretise_poisson_gpu_2(u_tilda, f, omega, N):
    '''
    Evaluate the linear matrix problem: A_tilda @ u_tilda = f, 
    without the need to define the matrix A_tilda. Boundary values
    are set to zero.
    
    Keyword arguments:
    u_tilda  -- ndarray of shape [(N-2)^2,], corresponding to
                the solution at each interior point
    f        -- ndarray of shape [(N-2)^2,], corresponding to
                the interior nodes
    
    '''
    # Define shared memory
    local_u = cuda.shared.array((haloSX, haloSY), numba.float32)
    
    # Local thread position
    i = cuda.threadIdx.x
    j = cuda.threadIdx.y
    
    # Global thread position
    pi, pj = cuda.grid(2)
    
    if pi >= N-2:
        return
    if pj >= N-2:
        return
    
    # Compute the vector index
    pk = pj * (N - 2) + pi
    
    # Load interior points of each block
    local_u[i + 1, j + 1] = u_tilda[pk]
    
    # Load halo boundries of each block
    if i == 0 and pi != 0:                          # Left edges
        local_u[i, j + 1] = u_tilda[pk - 1]
    if j == 0 and pj != 0:                          # Top edges             
        local_u[i + 1, j] = u_tilda[pk - (N - 2)]
    if i == SX - 1 and pi != (N - 2) - 1:           # Right edges
        local_u[-1, j + 1] = u_tilda[pk + 1]
    if j == SY - 1 and pj != (N - 2) - 1:           # Bottom edges
        local_u[i + 1, -1] = u_tilda[pk + (N - 2)]

    # Load global boundry conditions (boundry values = 0)
    if pi == 0: 
        local_u[i, j + 1] = numba.float32(0)        # Global left edge
    if pj == 0: 
        local_u[i + 1,j] = numba.float32(0)         # Global top edge
    if pi == (N - 2) - 1:
        local_u[i + 2, j + 1] = numba.float32(0)    # Global right edge
    if pj == (N - 2) - 1:
        local_u[i + 1, j + 2] = numba.float32(0)    # Global bottom edge

    cuda.syncthreads()
    
    # Calculation of interior nodes in f
    up = local_u[i + 2, j + 1]
    down = local_u[i, j + 1]
    left = local_u[i + 1, j]
    right = local_u[i + 1, j + 2]
    center = local_u[i + 1, j + 1]
    
    cuda.syncthreads()

    f[pk] = (N - 1)**2 * (numba.float32(4) * center - up - down - left - right) + (omega**2) * center
        

def eval_gpu_2(u_tilda):
    '''
    Evaluate the discrete Laplacian on the GPU.
    '''
    
    # Create empty array 
    f = np.empty((N-2)**2, dtype=np.float32)
    
    # Run GPU kernal
    nblocks = ((N-2) + 31) // 32
    discretise_poisson_gpu_2[(nblocks, nblocks), (32, 32)](u_tilda.astype('float32'), f, omega, N)
    
    return f.astype('float64')

The following does some timing comparisons:

In [6]:
for N in np.arange(500, 1000, 100):
    print(f'Time for kernal to evaluate with shared memory for N = {N}:')
    %timeit eval_gpu_2(u_tilda)

Time for kernal to evaluate with shared memory for N = 500:
2.41 ms ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate with shared memory for N = 600:
2.67 ms ± 46.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate with shared memory for N = 700:
2.83 ms ± 32 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate with shared memory for N = 800:
3.36 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate with shared memory for N = 900:
3.64 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
for N in np.arange(500, 1000, 100):
    print(f'Time for kernal to evaluate without shared memory for N = {N}:')
    %timeit eval_gpu(u.flatten())

Time for kernal to evaluate without shared memory for N = 500:
3.44 ms ± 42.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate without shared memory for N = 600:
3.75 ms ± 94.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate without shared memory for N = 700:
3.98 ms ± 8.52 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate without shared memory for N = 800:
4.24 ms ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Time for kernal to evaluate without shared memory for N = 900:
4.47 ms ± 45 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Hence, we see that for a range of different N values, the new GPU kernal offers faster evaluation time, which gets relatively faster and faster as N scales. This effectively shows that the issue with memory transfer is reduced, especially for large matricies (which scale with N^2, where memory transfer would increase more).